# Notebook for Selenium WebDriver

pip3 install selenium


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from os import path
import zipfile
import shutil
import xml.etree.ElementTree as ET


In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.close()

In [ ]:
test_url = 'http://tomcat.apache.org/'
driver.get(test_url)

## Searching for elements


In [ ]:
# search for a element h3 where the text contains a string of '8.5.57'
search_str = "//h3[contains(text(),'8.5.57')]"

In [ ]:
# search for an element where id is 'mainLeft'. Assumeably an id is unique throughout the doc
search_str = "//*[@id='mainLeft']"

In [ ]:
# search for an element where name is 'author'. Assumeably an name is unique throughout the doc
search_str = "//*[@name='author']"

In [ ]:
# search for an element where link is 'http://tomcat.apache.org/'
search_str = "//*[@href='http://tomcat.apache.org/']"

In [ ]:
# search for an element where class name is 'clearfix'
search_str = "//*[@class='clearfix']"

In [ ]:
# search for an element by tag name 'button'
search_str = "//button"

In [ ]:
# search for an element by a CSS selector, i.e., #xyz

In [ ]:
search_str = "//select[@name='Preferred']"

In [ ]:
# exact text search
search_str = "//option[text()='https://apache.cs.utah.edu/']"

## Spring Initializr website


In [ ]:
test_url = "https://start.spring.io/"
driver.get(test_url)

In [ ]:
driver.close()

In [ ]:
search_str = "//input[@id='input-group']"

## Scenario

```gherkin
Given I am at the Spring Initializer website "https://start.spring.io/"
When I create a Spring boot with the following attributes:
  | project | language | version | group          | artifact  | description | packaging | java_version |
  | Maven   | Kotlin   | 2.2.8   | com.drkiettran | my_kotlin | testing     | war       | 11           |
Then I should receive a zip file
And the file should contain a build.gradle file with version 2.2.8
```

### Description
I am using the startup website for Spring Boot projects.
In the example below, I basically create a propject with properties
defined above (in the When step).

When you click on 'Generate' button, a zip file is generated and 
download to your local disk. Here are few things to take notice:

- You need to provide a download directory or a default will be used.
- The name of the zip file is the 'artifact' + .zip
- Defining the dowlnoad folder is different between browser. I am using Chrome.

In [ ]:
test_url = "https://start.spring.io/"
download_dir = '/home/student/Downloads/selenium/'
artifact_name = 'my_kotlin'
download_file = download_dir + artifact_name + '.zip'

if path.exists(download_file):
    os.remove(download_file)

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option('prefs', {
        'download.default_directory': download_dir,
})

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(5)
driver.get(test_url)

# Choose Maven Project
search_str = "//*[text()='Maven Project']"
els = driver.find_elements_by_xpath(search_str)
els[0].click()

# Choose Kotlin
search_str = "//*[text()='Kotlin']"
els = driver.find_elements_by_xpath(search_str)
els[0].click()

# Choose SpringBoot version 2.2.8
search_str = "//*[text()='2.2.8']"
els = driver.find_elements_by_xpath(search_str)
els[0].click()

# Enter group name
search_str = 'input-group'
els = driver.find_elements_by_id(search_str)
els[0].clear()
els[0].send_keys('com.drkiettran')

# Enter artifact
search_str = 'input-artifact'
els = driver.find_elements_by_id(search_str)
els[0].clear()
els[0].send_keys(artifact_name)

# Choose War packaging
search_str = "//*[text()='War']"
els = driver.find_elements_by_xpath(search_str)
els[0].click()

# Choose Java 11
search_str = "//*[text()='11']"
els = driver.find_elements_by_xpath(search_str)
els[0].click()

# Click Generate
search_str = "//*[text()='Generate']"
els = driver.find_elements_by_xpath(search_str)
els[0].click()

# I should wait after the download and then process file there

In [ ]:
driver.close()

In [ ]:
tmp_dir = '/tmp/'
extracted_dir = tmp_dir + artifact_name + '/'

if path.exists(extracted_dir):
    shutil.rmtree(extracted_dir)

with zipfile.ZipFile(download_file, 'r') as zip_ref:
    zip_ref.extractall(tmp_dir)    

In [ ]:
pom_file = extracted_dir + 'pom.xml'
tree = ET.parse(pom_file)
root = tree.getroot()


In [ ]:
def get_spring_boot_version(root):
    for child in root:
        if 'parent' in child.tag:
            for gchild in child:            
                if 'version' in gchild.tag:
                    return gchild.text

print(get_spring_boot_version(root))                